### Now get optuna optimization trials working

TODO:
1. Improve early stopping such that I have control over patience parameter and ensure the _minimum_ validation loss is passed to the optuna study.

The most well-regularized hyperparameters for simple (`mse_loss` only) ended up being 
256 hidden features, 3.7995 first_omega_0, 2.9312 hidden_omega_0

With wavefunc loss, `squared_slowness` of around 0.5 may be close

```
Finished trial#26 with value: 0.09781524538993835 with parameters: 
{'first_omega_0': 4.839289222946841, 'hidden_omega_0': 13.756932872278343, 'squared_slowness': 0.27488941275825124, 'wave_loss_scale': 9.252313787089657e-08}
```

In [ ]:
import os
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet
from optuna.integration import PyTorchLightningPruningCallback
from surfbreak.studies import run_waveform_hyperparam_search, MetricsCallback
LOGDIR = '../tmp/testlogs'
MODELDIR = os.path.join(LOGDIR, 'opt_models')

def objective(trial):
    checkpoint_callback = pl.callbacks.ModelCheckpoint( # Filenames for each trial must be made unique
        os.path.join(MODELDIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss")
    tb_logger = pl.loggers.TensorBoardLogger(LOGDIR+'/', name="optuna")
    metrics_callback = MetricsCallback()     # Simple callback that saves metrics from each validation step.
    
    pl.seed_everything(42)
    trainer = pl.Trainer(logger=tb_logger, limit_val_batches=3,
                         max_epochs=2, 
                         gpus=1 if torch.cuda.is_available() else None,
                         callbacks=[metrics_callback],
                         early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss"),
                        )

    wavefunc_model = LitSirenNet(video_filepath='../data/shirahama_1590387334_SURF-93cm.ts',
                                 hidden_features=trial.suggest_categorical('hidden_features', [128, 256, 380]), #256,
                                 hidden_layers=trial.suggest_categorical('hidden_layers', [3]), #3,
                                 first_omega_0=trial.suggest_uniform('first_omega_0', 0.5, 2.5), #1.5, 
                                 hidden_omega_0=trial.suggest_uniform('hidden_omega_0', 8.5, 15), #11.7,
                                 squared_slowness=trial.suggest_uniform('squared_slowness', 0.20, 0.70), #0.23,
                                 steps_per_vid_chunk=10, 
                                 learning_rate=1e-4, grad_loss_scale=0, 
                                 wavefunc_loss_scale=trial.suggest_loguniform('wavefunc_loss_scale', 5e-9, 1e-7), #2e-8,
                                 wavespeed_loss_scale=trial.suggest_loguniform('wavespeed_loss_scale', 1e-4, 1e-2), #1e-3,
                                 xrange=(0,200), timerange=(0,3*10), chunk_duration=10, chunk_stride=10)
                                 # With default settings wavezone image dimensions are (y=139, x=1515)

    trainer.fit(wavefunc_model)
    return metrics_callback.metrics[-1]["val_loss"].item()


study = run_waveform_hyperparam_search(objective, n_trials=3, timeout=2*60, model_dir=MODELDIR, 
                                       prune=False, n_startup_trials=2, n_warmup_steps=5)


In [ ]:
study.best_params

In [ ]:
sdf = study.trials_dataframe()
top_trials = sdf.sort_values(by='value')[:5]
top_trials

In [ ]:
print(top_trials.mean())
print(top_trials.std())